# Polars Tutorial and Demonstration
> ### Jonathan Scofield
#### This notebook will help you set up polars on you computer and query a large CSV file. For more information about Polars, please visit the official [website](https://pola.rs/). <br>
#### We will be using real estate sales data from the State of Connecticut for property valued $2K or more from the year 2001 to 2021.
#### The data used for this project is in the public domain and is can be found [here](https://catalog.data.gov/dataset/real-estate-sales-2001-2018).

## Setup <br>
#### You need to have a version of Python installed equal to or greater than 3.10.
#### Use this command to install Polars:
>pip install 'polars[all]' 

## Scanning a CSV

#### Import required modules.

In [14]:
# "pl" is the conventional alias for the Polars library
import polars as pl
import os

#### Let's take a look at the size of the CSV file we want to examine.

In [15]:
f"{round(os.path.getsize('Real_Estate_Sales_2001-2021_GL.csv') / (1024 ** 2), 2)} MB"

'113.24 MB'

#### It is fairly large, so we will scan it to a LazyFrame.

In [16]:
df = pl.scan_csv( #We are scanning, not reading
    'Real_Estate_Sales_2001-2021_GL.csv', 
    ignore_errors = False,   # No crash on error
    infer_schema_length = 1000, # Sample size for schema detection
    low_memory = True, # Memory > speed
    try_parse_dates = True, # Format dates automatically   
)

#### After scanning, we can see the proposed schema from the given sample size:

In [17]:
df.schema # View the inferred schema

{'Serial Number': Int64,
 'List Year': Int64,
 'Date Recorded': Date,
 'Town': Utf8,
 'Address': Utf8,
 'Assessed Value': Float64,
 'Sale Amount': Float64,
 'Sales Ratio': Float64,
 'Property Type': Utf8,
 'Residential Type': Utf8,
 'Non Use Code': Utf8,
 'Assessor Remarks': Utf8,
 'OPM remarks': Utf8,
 'Location': Utf8}

#### Because this is a LazyFrame, Polars can't return the count of rows.

In [18]:
df.select(pl.count()) # This won't return anything on a LazyFrame

<LazyFrame [1 col, {"count": UInt32}] at 0x7FB0105503A0>

## Querying the Data

#### We can query the data as if it was a SQL database using the following syntax:

In [19]:
select_df = pl.SQLContext(register_globals = True).execute(
   ''' 
   SELECT
        *
    FROM 
        df
    LIMIT 5
    '''
)

#### To view the data, we must call the **collect()** method.

In [20]:
head_df = select_df.collect(streaming = True) # Perform query and load into memory
head_df

Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
i64,i64,date,str,str,f64,f64,f64,str,str,str,str,str,str
2020348,2020,null,"""Ansonia""","""230 WAKELEE AV…",150500.0,325000.0,0.463,"""Commercial""",null,null,null,null,null
20002,2020,2020-02-10,"""Ashford""","""390 TURNPIKE R…",253000.0,430000.0,0.5883,"""Residential""","""Single Family""",null,null,null,null
210317,2021,2022-05-07,"""Avon""","""53 COTSWOLD WA…",329730.0,805000.0,0.4096,"""Residential""","""Single Family""",null,null,null,"""POINT (-72.846…"
200212,2020,2021-09-03,"""Avon""","""5 CHESTNUT DRI…",130400.0,179900.0,0.7248,"""Residential""","""Condo""",null,null,null,null
200243,2020,null,"""Avon""","""111 NORTHINGTO…",619290.0,890000.0,0.6958,"""Residential""","""Single Family""",null,null,null,null


#### Once a LazyFrame has been collected, we can access the data.

In [21]:
head_df.select(pl.count()) # Count number of rows

count
u32
5


#### We can perform most basic SQL queries on the data. Let's try some string manipulation:

In [22]:
concat_df = pl.SQLContext(register_globals = True).execute(
   ''' 
   SELECT
       "Town",
       "Residential Type",
       upper(trim("Town")) + '-' + upper(trim("Residential Type")) as "New Column",
       "Assessed Value"
    FROM 
        df
    WHERE 
        "List Year" = 2021 AND 
        "Assessed Value" > 1000000 
        AND "Residential Type" IS NOT NULL
    ORDER BY 
        "Assessed Value" DESC
   LIMIT 5
    '''
).collect(streaming = True)
concat_df

Town,Residential Type,New Column,Assessed Value
str,str,str,f64
"""New Canaan""","""Condo""","""NEW CANAAN-CON…",3.791354e7
"""New Canaan""","""Condo""","""NEW CANAAN-CON…",3.791354e7
"""New Canaan""","""Condo""","""NEW CANAAN-CON…",3.791354e7
"""Darien""","""Single Family""","""DARIEN-SINGLE …",3.50301e7
"""Darien""","""Single Family""","""DARIEN-SINGLE …",3.50301e7


In [23]:
total_value_df = pl.SQLContext(register_globals = True).execute(
   ''' 
   SELECT
       UPPER(TRIM("Town")),
       UPPER(TRIM("Property Type")),
       SUM("Assessed Value") AS "Total Assessed Value"
    FROM 
        df
    WHERE 
        "List Year" = 2021 AND 
        "Town" IS NOT NULL AND
        "Property Type" IS NOT NULL
    GROUP BY 
        "Town", 
        "Property Type"
    ORDER BY
       "Total Assessed Value" DESC,
    LIMIT 10
    '''
).collect(streaming = True)
total_value_df

Town,Property Type,Total Assessed Value
str,str,f64
"""BRIDGEPORT""","""COMMERCIAL""",7.6099608e8
"""STAMFORD""","""RESIDENTIAL""",7.4672812e8
"""WESTPORT""","""RESIDENTIAL""",6.27301505e8
"""STAMFORD""","""COMMERCIAL""",6.08599591e8
"""NORWALK""","""RESIDENTIAL""",5.71108576e8
"""NEW CANAAN""","""RESIDENTIAL""",5.535687e8
"""FAIRFIELD""","""RESIDENTIAL""",5.170931e8
"""DARIEN""","""RESIDENTIAL""",4.9873546e8
"""STAMFORD""","""APARTMENTS""",3.0715507e8


In [24]:
best_premium_df = pl.SQLContext(register_globals = True).execute(
   ''' 
   SELECT
        TRIM(UPPER("Town")),
        "Address",
       ("Sale Amount" - "Assessed Value") AS "Premium"
    FROM 
        df
    WHERE 
        "List Year" = 2021 AND 
        "Town" IS NOT NULL
    ORDER BY
       "Premium" DESC
    LIMIT 1
    '''
).collect(streaming = True)
best_premium_df

Town,Address,Premium
str,str,f64
"""STAMFORD""","""695 EAST MAIN …",2.091e8


## Writing Data

#### Since we usually use Polars when dealing with large files, it can be helpful to use it to compress data into smaller files.

#### Here is how to compress the CSV file into a parquet file:

In [25]:
df.collect(streaming = True).write_parquet('Real_Estate_Sales_2001-2021_GL.parquet')

#### Finally, we can check the size of the parquet file to compare:

In [26]:
f"{round(os.path.getsize('Real_Estate_Sales_2001-2021_GL.parquet') / (1024 ** 2), 2)} MB"

'33.52 MB'